In [274]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [294]:
import csv
import pandas as pd
import numpy as np

In [295]:
#зчитування напряму
pathToData = '/gdrive/MyDrive/int20h_hackatone/data_analytics.csv'
with open(pathToData, newline='') as f:
    reader = csv.DictReader(f)

#зчитування як DataFrame
records = pd.read_csv(pathToData, delimiter=',')
records;

In [296]:
#test - кількість місяців підписки включно з тріалом
records.groupby('Subscriber ID').count()[['Event Date']]

,Event Date
Subscriber ID,
223411322456765433,6
223411322456765434,6
223411322456765435,6
223411322456765437,6
223411322456765438,6
...,...
223411322456766800,1
223411322456766801,1
223411322456766802,1


In [297]:
#test - кількість місяців підписки включно з тріалом - конкретний юзер
subCount = records['Subscriber ID'].value_counts()
subCount[223411322456765437]

6

In [298]:
#тест
records[(records['Subscription Offer Type'] == 'Free Trial') & (records['Subscriber ID'] == 223411322456766566)]

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
833,2019-07-20,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1


In [299]:
#тест
freeTrial = records['Subscription Offer Type'].isnull()
subId = records['Subscriber ID'] == 223411322456766566
records[freeTrial & subId]

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
1391,2019-07-27,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
1769,2019-08-03,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
2059,2019-08-10,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
2258,2019-08-17,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
2375,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1


In [ ]:
#test
#підписки по датах, згруповані по юзеру
#freeTrial = records['Subscription Offer Type'].isnull()
for record in records['Subscriber ID'].unique():
  subId = records['Subscriber ID'] == record
  print(records[subId][['Event Date', 'Subscription Offer Type', 'Subscriber ID']])
  #print(records[freeTrial & subId][['Event Date', 'Subscription Offer Type', 'Subscriber ID']])

In [301]:
#унікальні ід - кількість юзерів унікальних
uniqueUsersCount = len(records['Subscriber ID'].unique())
print(records['Subscriber ID'].unique()) 
uniqueUsersCount

[223411322456765433 223411322456765434 223411322456765435 ...
 223411322456766802 223411322456766805 223411322456766808]


1019

In [ ]:
#юзери, які продовжили підписку
subCount = records['Subscriber ID'].value_counts()
for sub in records['Subscriber ID'].unique():
  if(subCount[sub] > 1):
    print(sub, subCount[sub])

In [303]:
#юзери, які не оформили підписку, після тріалу
trialUsersCount = 0
subCount = records['Subscriber ID'].value_counts()
for sub in records['Subscriber ID'].unique():
  if(subCount[sub] == 1):
    trialUsersCount += 1
print(trialUsersCount)

520


In [304]:
#юзери по кількості тижнів підписки
subsPerWeek = { 
  '1 weeks': 0, 
  '2 weeks': 0, 
  '3 weeks': 0, 
  '4 weeks': 0,
  '5 weeks': 0,
  '6 weeks': 0,
  }

weekCount = 0
weekSubCount = records['Subscriber ID'].value_counts()
for sub in records['Subscriber ID'].unique():
  week = weekSubCount[sub]
  weekName = '{} weeks'.format(week)
  subsPerWeek[weekName] += 1

#if you want to rename key
#newKey ='1 week (trial only)'
#subsPerWeek[newKey] = subsPerWeek.pop('1 weeks')

subsPerWeekDF = pd.DataFrame(subsPerWeek, index=[0]) 
subsPerWeekDF

,1 weeks,2 weeks,3 weeks,4 weeks,5 weeks,6 weeks
0,520,146,91,107,63,92


In [305]:
#обрахунок LTV
LTVdict = { 
  'Subscriber ID': [], 
  'SubTime': [], 
  'TotalSpend': [], #total spend by user for all time
  'LTV': [], #формула з завдання, як я зрозумів
  'CTV': [], #customer time value = загальний прибуток з юзера / час підписки. (залежить від тримвалості підписки). 0, якщо юзер не оформив платну підписку
  }

subscriptionPrice = 9.99
#store takes 30%
margin = 0.7
devProceeds = subscriptionPrice * margin

#юзери, які продовжили підписку
subCount = records['Subscriber ID'].value_counts()
for sub in records['Subscriber ID'].unique():
  weeks = subCount[sub] #weeks of subsciption
  totalSpend = devProceeds * (weeks - 1)
  ltv = 0
  for week in range(weeks):
    #їх формула LTV, як я зрозумів
    ltv += devProceeds * week 
  LTVdict['Subscriber ID'].append(sub)
  LTVdict['SubTime'].append(weeks)
  LTVdict['TotalSpend'].append(totalSpend)
  LTVdict['LTV'].append(ltv)
  LTVdict['CTV'].append(totalSpend/weeks)

df = pd.DataFrame(LTVdict) 
pathToSaveLTVperUser = '/gdrive/MyDrive/int20h_hackatone/LTV_per_user.csv'
df.to_csv(pathToSaveLTVperUser, index = False)

df

,Subscriber ID,SubTime,TotalSpend,LTV,CTV
0,223411322456765433,6,34.965,104.895,5.8275
1,223411322456765434,6,34.965,104.895,5.8275
2,223411322456765435,6,34.965,104.895,5.8275
3,223411322456765437,6,34.965,104.895,5.8275
4,223411322456765438,6,34.965,104.895,5.8275
...,...,...,...,...,...
1014,223411322456766800,1,0.000,0.000,0.0000
1015,223411322456766801,1,0.000,0.000,0.0000
1016,223411322456766802,1,0.000,0.000,0.0000
1017,223411322456766805,1,0.000,0.000,0.0000


In [306]:
ChurnTotal = ((subsPerWeek['1 weeks']-subsPerWeek['6 weeks'])/subsPerWeek['1 weeks'])*100
ChurnPaid = ((subsPerWeek['2 weeks']-subsPerWeek['6 weeks'])/subsPerWeek['2 weeks'])*100
print('Churn for all weeks:', ChurnTotal)
print('Churn for paid weeks:', ChurnPaid)
ALTtotal = 100/ChurnTotal
ALTpaid = 100/ChurnPaid
print('Average customer lifespan in months (ALT) all weeks', ALTtotal)
print('Average customer lifespan in months (ALT) paid weeks', ALTpaid)

Churn for all weeks: 82.3076923076923
Churn for paid weeks: 36.986301369863014
Average customer lifespan in months (ALT) all weeks 1.2149532710280373
Average customer lifespan in months (ALT) paid weeks 2.7037037037037037


In [307]:
#average number of paid transactions (T) :
totalWeeks = 6
paidWeeks = 5 #усього тижнів 6, але перший безкоштовний

Ttotal = len(records[records['Subscription Offer Type'].isnull()]) / totalWeeks #isnull() бо це транзакції підпіски платної
Tpaid = len(records[records['Subscription Offer Type'].isnull()]) / paidWeeks #isnull() бо це транзакції підпіски платної

#CLV (Customer lifetime value) total
CLVtotal = (Ttotal * devProceeds * ALTtotal)/uniqueUsersCount
#CLV (Customer lifetime value) for paid weekstotal
CLVpaid = (Tpaid * devProceeds * ALTpaid)/(uniqueUsersCount - trialUsersCount)

ARPU = df.TotalSpend.sum()/uniqueUsersCount
ARPUwithoutTrial = df.TotalSpend.sum()/(uniqueUsersCount - trialUsersCount)
TotalLTV = ARPU / ALTtotal
TotalLTVpaid = ARPU / ALTpaid

In [310]:
totalValues = { 
    'TotalLTV': TotalLTV, 
    'TotalLTVpaid': TotalLTVpaid, 
    'CLVtotal': CLVtotal, 
    'CLVpaid': CLVpaid, 
    'ARPUtotal': ARPU, 
    'ARPUpaid': ARPUwithoutTrial, 
    'UniqueUsersCount': uniqueUsersCount, 
    'TrialOnlyUsersCount': trialUsersCount, 
    'SubscribersCount(paid)': (uniqueUsersCount-trialUsersCount), 
    'ChurnTotal': ChurnTotal, 
    'ChurnPaid': ChurnPaid,
    'ALTtotal': ALTtotal,
    'ALTpaid': ALTpaid,
  }

totalValuesDF = pd.DataFrame(totalValues, index=[0]) 
pathToSaveTotalLTV = '/gdrive/MyDrive/int20h_hackatone/LTV_total.csv'
totalValuesDF.to_csv(pathToSaveTotalLTV, index = False)

totalValuesDF

,TotalLTV,TotalLTVpaid,CLVtotal,CLVpaid,ARPUtotal,ARPUpaid,UniqueUsersCount,TrialOnlyUsersCount,SubscribersCount(paid),ChurnTotal,ChurnPaid,ALTtotal,ALTpaid
0,7.687549,3.454525,1.89128,10.313598,9.340013,19.073092,1019,520,499,82.307692,36.986301,1.214953,2.703704


In [311]:
#по 2 варіанти розрахунків, бо незрозуміло, як треба враховувати пробний тиждень

print('ARPU for all users:', ARPU)
print('ARPU without TrialOnly users:', ARPUwithoutTrial)
print('CLV for all users:', CLVtotal)
print('CLV without TrialOnly users:', CLVpaid)
print('Total LTV for all users:', TotalLTV)
print('Total LTV without TrialOnly users:', TotalLTVpaid)

ARPU for all users: 9.340012757605495
ARPU without TrialOnly users: 19.073092184368733
CLV for all users: 1.8912798418827326
CLV without TrialOnly users: 10.313597995991984
Total LTV for all users: 7.687548962029139
Total LTV without TrialOnly users: 3.454525266511621
